In [11]:
# 傳送 text 資料，附帶檔案 (此方法對 office outlook 2016 中文不能用)
import pandas as pd
import numpy as np
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
#
fromAddr = "lanjyh@cht.com.tw"
myPass = "@@zzxx5413"
toAddr = "hw2050@cht.com.tw"
#
msg = MIMEMultipart()
msg['From'] = fromAddr
msg['To'] = toAddr
msg['Subject'] = "2021/6/2 嘉義行分個人保管財產清單"
#
body = '''
各位同仁:大家好

本郵件由程式自動發送，不用回信。

附件為您所保管的財產資料，預定每月將清查乙次。

請各位同仁留意本身所負責保管之財物，並定期更正相關資料。

期望財產管理業務能更上軌道運作 !!!


藍志祥敬上
'''
# add text 
msg.attach(MIMEText(body, 'plain'))

# attach a file
part = MIMEApplication(open('財產.xlsx','rb').read()) 
part.add_header('Content-Disposition', 'attachment', filename= "財產1.xlsx")
msg.attach(part) 
server = smtplib.SMTP('webmail.cht.com.tw', 25)
server.login(fromAddr, myPass)
server.send_message(msg)
server.quit()

(221, b'ms25.cht.com.tw')

In [1]:
# 將 property data 與 include mailaddress 送出 (目前使用正確版本)
import pandas as pd
import numpy as np
import smtplib
import shutil
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

def send_to_mail(to_Addr, send_file_name):
    fromAddr = "lanjyh@cht.com.tw"
    myPass = "@@zzxx5413"   # input password
    toAddr = to_Addr
    #
    msg = MIMEMultipart()
    msg['From'] = fromAddr
    msg['To'] = to_Addr
    msg['Subject'] = "2021/07/01 嘉義行分個人保管財產清單"    # input date
    #
    body = '''
    各位同仁:大家好，此為 110 年 07 月份財產清單

    本郵件由程式自動發送，不用回信。

    附件為您所保管的財產資料，預定每月將清查乙次。

    請各位同仁留意本身所負責保管之財物，並定期更正相關資料。

    期望財產管理業務能更上軌道運作 !!!


    藍志祥敬上
    '''
    # add text 
    msg.attach(MIMEText(body, 'plain'))

    # attach a file
    part = MIMEApplication(open(send_file_name,'rb').read()) 
    part.add_header('Content-Disposition', 'attachment', filename=send_file_name) 
    msg.attach(part) 

    server = smtplib.SMTP('webmail.cht.com.tw', 25)
    server.login(fromAddr, myPass)
    server.send_message(msg)
    server.quit()
#---------------讀取資料檔-----------------
# include mailaddress
file_name = "mail_address.csv"
file_path = "./{}".format(file_name)
csv_df = pd.read_csv(file_path)
csv_df = csv_df[csv_df['姓氏'].str.contains("2-")]
csv_df = csv_df.dropna(axis=1)
csv_df = csv_df.drop(labels = ["行動電話", "生日", "性別", "紀念日", "敏感度", "電子郵件類型","電子郵件顯示名稱", "優先順序", "縮寫"], axis=1)
csv_df['姓氏'] = csv_df['姓氏'].str.replace('2-','').str.replace('工程師','').str.replace('股長','')

#---------------讀取資料檔---------------------
# include property data
file_name = "form(2021-07-01).xls"     # input source data
file_path = "./data_property_form/{}".format(file_name)
xls_df = pd.read_excel(file_path)
values = {'異動者':'未指配'}
xls_df = xls_df.fillna(value=values)
xls_df['異動者'] =  xls_df['異動者'].str.strip()

# merge two dataframe
xls_df = pd.merge(xls_df, csv_df, how='inner', left_on="異動者", right_on="姓氏")
owners = list(xls_df['異動者'].unique())
owners_address = list(xls_df['電子郵件地址'].unique())
xls_df = xls_df.drop(labels = ["姓氏", "電子郵件地址"], axis=1)

# 將所屬資料 mail_out

file_names = ["property({}).xlsx".format(address) for address in owners_address]
file_paths = ["{}".format(filename) for filename in file_names]

for name, path, mail in zip(owners,file_paths,owners_address):
    xls_df[xls_df['異動者'] == name].to_excel(path,sheet_name = 'Sheet1',index=False) 
    send_to_mail(mail, path)

for remove_file in file_names:
    source = r'./{}'.format(remove_file)
    destination = r'./results(email)/{}'.format(remove_file) 
    shutil.move(source,destination)

WARNING *** file size (16108770) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
